In [1]:
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np

# Define paths
data_dir = "data"
infected_dir = os.path.join(data_dir, "infected")
uninfected_dir = os.path.join(data_dir, "uninfected")

class CellDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Convert .tif to RGB
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Get all file paths and labels
infected_files = [os.path.join(infected_dir, f) for f in os.listdir(infected_dir) if f.endswith('.tif')]
uninfected_files = [os.path.join(uninfected_dir, f) for f in os.listdir(uninfected_dir) if f.endswith('.tif')]

infected_labels = [1] * len(infected_files)  # Label for infected
uninfected_labels = [0] * len(uninfected_files)  # Label for uninfected

all_files = infected_files + uninfected_files
all_labels = infected_labels + uninfected_labels

# Split into train and validation
train_files, val_files, train_labels, val_labels = train_test_split(all_files, all_labels, test_size=0.2, stratify=all_labels, random_state=42)

# Define transformations (same as original code)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet input size
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
])

# Create dataset objects
train_dataset = CellDataset(train_files, train_labels, transform=transform)
val_dataset = CellDataset(val_files, val_labels, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=42, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=42, shuffle=False)

# Load pre-trained ResNet50
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Identity()  # Remove the final classification layer

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)

# Feature extraction function
def extract_features(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    features = []
    labels = []
    with torch.no_grad():  # Disable gradient calculations
        for inputs, label in dataloader:
            inputs = inputs.to(device)
            feature = model(inputs)  # Extract features
            features.append(feature.cpu().numpy())
            labels.extend(label)

    # Convert the list of features into a numpy array
    features = np.vstack(features)
    return features, np.array(labels)

# Extract features from training and validation sets
train_features, train_labels = extract_features(resnet50, train_loader)
val_features, val_labels = extract_features(resnet50, val_loader)

# Train an XGBoost model
dtrain = xgb.DMatrix(train_features, label=train_labels)
dval = xgb.DMatrix(val_features, label=val_labels)

# Set XGBoost parameters
params = {
    "objective": "binary:logistic",  # Binary classification
    "eval_metric": "logloss",        # Log loss for binary classification
    "max_depth": 6,                  # Maximum depth of the tree
    "learning_rate": 0.1,            # Learning rate
    "n_estimators": 100              # Number of trees
}

# Train the model
model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10)

# Save the XGBoost model
model.save_model("xgboost_cancer_model.json")

# Make predictions and evaluate
def evaluate_xgboost(model, dval):
    preds = model.predict(dval)
    predictions = (preds > 0.5).astype(int)  # Convert to binary prediction (0 or 1)
    accuracy = (predictions == dval.get_label()).mean() * 100
    print(f"Validation Accuracy: {accuracy:.2f}%")

evaluate_xgboost(model, dval)


/home/hausemaster/.local/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [13:07:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	eval-logloss:0.66913
[1]	eval-logloss:0.64807
[2]	eval-logloss:0.61493
[3]	eval-logloss:0.60042
[4]	eval-logloss:0.58379
[5]	eval-logloss:0.57031
[6]	eval-logloss:0.55531
[7]	eval-logloss:0.54034
[8]	eval-logloss:0.53097
[9]	eval-logloss:0.52014
[10]	eval-logloss:0.51738
[11]	eval-logloss:0.51111
[12]	eval-logloss:0.50378
[13]	eval-logloss:0.49531
[14]	eval-logloss:0.48867
[15]	eval-logloss:0.48123
[16]	eval-logloss:0.47457
[17]	eval-logloss:0.46803
[18]	eval-logloss:0.46915
[19]	eval-logloss:0.45974
[20]	eval-logloss:0.45691
[21]	eval-logloss:0.45343
[22]	eval-logloss:0.45378
[23]	eval-logloss:0.44828
[24]	eval-logloss:0.44935
[25]	eval-logloss:0.44411
[26]	eval-logloss:0.44043
[27]	eval-logloss:0.43910
[28]	eval-logloss:0.44051
[29]	eval-logloss:0.44246
[30]	eval-logloss:0.43887
[31]	eval-logloss:0.44014
[32]	eval-logloss:0.43832
[33]	eval-logloss:0.43924
[34]	eval-logloss:0.43723
[35]	eval-logloss:0.43598
[36]	eval-logloss:0.43625
[37]	eval-logloss:0.43859
[38]	eval-logloss:0.43